<a href="https://colab.research.google.com/github/zsevall/DSAI301/blob/main/Zeynep_Seval_fall2024_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# zeynep_seval_2024660042_FINAL EXAM

**Question 1**
Web Search Engine

# **Question 1a**

Part 1> Finding Data

The flow of execution is:
*   Takes a seed URL crawlWeb(seed)
*   Get the page content using getPage
*   Find all links on that page using get_all_links (which uses get_next_target)
*   Visits each linked page recursively and Adds new links to our to-crawl list using union

In [ ]:
# From Midterm results

def getPage(url): # This function does the actual work of fetching web pages:
  try:
    import urllib.request
    page = urllib.request.urlopen(url).read()
    page = page.decode("utf-8")
    return page
  except:
    return ""  # If anything goes wrong, return empty string



def get_next_target(page):   # This is our link finder. It looks through HTML in the given page to find the next page/link:
  start_link = page.find('<a href=')        # Find the start of a link tag
  if start_link == -1:                      # If no link found
    return None, 0                          # We're done here
  start_quote = page.find('"', start_link)  # Find the first quote of the URL
  end_quote = page.find('"', start_quote+1) # Find the ending quote
  url = page[start_quote + 1:end_quote]     # Extract the URL
  return url, end_quote                     # Return URL
  # we return here end_quote as well because we will do a trick in main function, cut the page from that point



def get_all_links(page):  # This function uses get_next_target repeatedly to find all links on a page and adds them into a list called links:
  links = []                        # Start with empty list
  while True:
    url, endpos = get_next_target(page)  # Find next link
    if url:                              # If we found one
      links.append(url)                  # Add it to our list
      page = page[endpos:]              # Move to rest of page
    else:
      break                             # No more links? We're done
  return links


# Helper function union which helps us add new URLs to our to-crawl list without duplicates
# it will be used in Crawlweb function later

def union(list1,list2):
  for scoop in list2:              # for each item in the list
    if scoop not in list1:         # If we haven't seen it before
      list1.append(scoop)          # add it into list
  return list1

Part 2> Building Index

The flow of execution is:
*  Cleans HTML content (get_clean_page)
*  Builds a searchable index (add_to_index, add_page_to_index)

In [ ]:
# From HW5 Answers

def getclearpage(content): # The HTML cleaning function:
    # Extract title
    title = content[content.find("<title>")+7:content.find("</title>")]
    # Extract body
    body = content[content.find("<body>")+6:content.find("</body>")]

    # Remove HTML tags from body
    while body.find(">") != -1:  # While there are still HTML tags
        start = body.find("<")   # Find start of tag
        end = body.find(">")     # Find end of tag
        # Remove the tag by taking everything before "<" and after ">"
        body = body[:start] + body[end+1:]

    # Return combined clean text
    return title + body

def add_to_index(index, keyword, url):
  if keyword in index:
    index[keyword].append(url)
  else:
    index[keyword] = [url]

def addPageToIndex(index, url, content):
  content = getclearpage(content)
  words = content.split()
  for word in words:
    add_to_index(index, word, url)


Part 3> Ranking

The flow of execution is:
*  Generates a graph indicating the interconnections between web pages while crawling the web (crawlWeb)
*   Defines the ranking (computeRanks)
*  Provides search functionality (lookup)

In [ ]:
# From Lecture Notes Modified crawlWeb function for Ranking
# The graph shows the structure of the web pages and their connections.
# Each key in the graph is a web page (URL), and its value is a list of outlinks (other pages it links to).
# This structure is used to model the relationships between pages, which can be analyzed for ranking and crawling.

def crawlWeb(seed):
    tocrawl = [seed]
    crawled = []
    index = {}
    graph = {}  # Added to Initialize the graph dictionary for printing purpose in 1b

    while tocrawl:
        page = tocrawl.pop()
        if page not in crawled:
            content = getPage(page)
            outlinks = get_all_links(content)  # Added to Get all outbound links
            graph[page] = outlinks             # Adds the page and its outlinks to the graph
            addPageToIndex(index, page, content)
            union(tocrawl, outlinks)
            crawled.append(page)

    return index, graph  # Return both index and graph

# **Question 1b**

In [ ]:
def printGraph(graph): # print everything in required format
    print("The graph has", len(graph), "elements:")
    count = 1
    for page, outlinks in graph.items():
        print(f"\t{count}. [{page}]: {outlinks}")
        count += 1

# Execution
seed_url = "https://searchengineplaces.com.tr/"
index, graph = crawlWeb(seed_url)
printGraph(graph)

The graph has 10 elements:
	1. [https://searchengineplaces.com.tr/]: ['http://www.searchengineplaces.com.tr/travel_guide.html']
	2. [http://www.searchengineplaces.com.tr/travel_guide.html]: ['http://www.searchengineplaces.com.tr/ankara.html', 'http://www.searchengineplaces.com.tr/konya.html', 'http://www.searchengineplaces.com.tr/istanbul.html', 'http://www.searchengineplaces.com.tr/oktayrecommends.html', 'http://www.searchengineplaces.com.tr/seymarecommends.html']
	3. [http://www.searchengineplaces.com.tr/seymarecommends.html]: ['http://www.searchengineplaces.com.tr/oktayrecommends.html', 'http://www.searchengineplaces.com.tr/konya.html']
	4. [http://www.searchengineplaces.com.tr/oktayrecommends.html]: ['http://www.searchengineplaces.com.tr/istanbul.html']
	5. [http://www.searchengineplaces.com.tr/istanbul.html]: ['http://www.searchengineplaces.com.tr/maidens_tower.html', 'http://www.searchengineplaces.com.tr/galata_tower.html']
	6. [http://www.searchengineplaces.com.tr/galata_tower.h

# **Question 1c**

In [ ]:
# From Lecture Notes
def computeRanks(graph): # This function takes a graph dictionary as input where keys are pages and values are lists of outlinks:

    d = 0.8                               # Dampening factor (probability of following a link vs. random jump)
    N = len(graph)                        # Total number of pages in the graph
    numloops = 10                         # Number of iterations to run the algorithm (more loops = more accurate)
    ranks = {}                            # Initialize empty dictionary to store page ranks

    for page in graph:                    # Initial setup: give each page an equal rank
        ranks[page] = 1/N                 # Each page starts with rank 1/N (equal distribution)

    for i in range(0, numloops):          # Main iteration loop to refine ranks
        newranks = {}                     # Initialize empty dictionary for new rank values

        for page in graph:                # For each page in the graph
            newrank = (1-d)/N             # Start with the random probability

            for node in graph:            # Find pages that link to this page
                if page in graph[node]:   # If this node links to our page


                    newrank = newrank + d*(ranks[node]/len(graph[node]))  # Add rank contribution from this inbound link:
                                                                          # (node's rank * dampening factor) / (number of outbound links from node)
            newranks[page] = newrank      # Store the calculated new rank for this page

        ranks = newranks                  # Update the ranks dictionary with new values

    return newranks                       # Return the final computed ranks


In [ ]:
ranks = computeRanks(graph) # Compute ranks using the function

In [ ]:
def printRanks(ranks): # helper To print everything in required format
    for page, rank in ranks.items():
        print(f"The rank of the page {page} : {rank:.2f}")  # Print rank with 2 decimal places for clarity

# Execution
printRanks(ranks)


The rank of the page https://searchengineplaces.com.tr/ : 0.02
The rank of the page http://www.searchengineplaces.com.tr/travel_guide.html : 0.15
The rank of the page http://www.searchengineplaces.com.tr/seymarecommends.html : 0.07
The rank of the page http://www.searchengineplaces.com.tr/oktayrecommends.html : 0.07
The rank of the page http://www.searchengineplaces.com.tr/istanbul.html : 0.17
The rank of the page http://www.searchengineplaces.com.tr/galata_tower.html : 0.09
The rank of the page http://www.searchengineplaces.com.tr/maidens_tower.html : 0.09
The rank of the page http://www.searchengineplaces.com.tr/konya.html : 0.07
The rank of the page http://www.searchengineplaces.com.tr/mevlana.html : 0.05
The rank of the page http://www.searchengineplaces.com.tr/ankara.html : 0.04


# **Question 1d**

In [ ]:
def rankedLookup(index, key, graph):
    if key not in index:
        return []

    # Calculate ranks
    ranks = computeRanks(graph)

    # remove duplicates
    urls = index[key]
    unique_urls = []
    for url in urls:
        # Only add URL if we haven't added already
        if url not in unique_urls:
            unique_urls.append(url)

    # Sort unique URLs by their rank
    sorted_urls = sorted(unique_urls, key=lambda x: ranks[x], reverse=True) # ALLOWED BUILT-IN FUNCTION
    return sorted_urls

In [ ]:
# Test rankedLookup
index, graph = crawlWeb(seed_url)
results = rankedLookup(index, "in", graph)
for result in results:
    print(result)

http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html


# **Question 1e**

Modify the original lookup function
to optionally perform page ranking. This modification will be as follows:
If the procedure is given 2 inputs, specifically an index and a key, it will function like
the original lookup. However, if it’s given 4 inputs, namely index, key, graph, and
computing procedure, it will perform page ranking using the provided graph and computing procedure. In case of 3 inputs, it will raise a warning (not an error!) as specified
below.


Notice that both lookup with ranking option and rankedLookup do not provide the same
URL multiple times in their outputs, as they are designed to rank URLs, resulting in each URL
appearing only once. Make sure whatever parameter you use while calling the procedure, each
url should be diplayed only once

In [ ]:
def lookup(index, key, graph=None, computing_procedure=None):
    # an output string to collect results so that only one print is used
    output = ""

    # Check if the key exists in the index
    if key not in index:
        output += f"No results found for key: {key}\n"
    else:

        # remove duplicates
        urls = index[key]
        unique_urls = []
        for url in urls:
            # Only add URL if we haven't added already
            if url not in unique_urls:
               unique_urls.append(url)

        # standard lookup
        if graph is None:
            for url in unique_urls:
                output += url + "\n"

        # lookup with ranking, SAME AS RANKED LOOKUP
        elif computing_procedure:
            ranks = computing_procedure(graph)
            sorted_urls = sorted(unique_urls, key=lambda x: ranks[x], reverse=True)  # ALLOWED BUILT-IN FUNCTION
            for url in sorted_urls:
                output += url + "\n"

        # lookup with warning
        else:
            output += "Warning: No ranking procedure given\n"
            for url in unique_urls:
                output += url + "\n"

    # Print all collected outputs at once
    print(output)  # Single print statement


In [ ]:
# Test
lookup(index, "in")

http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html



In [ ]:
# Test
lookup(index, "in", graph)


http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html



In [ ]:
# Test
lookup(index, "in", graph, computing_procedure=computeRanks)

http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html



**Question 2 - Wordle Game Solver**

As you know, in the midterm exam, a problem involving playing the Wordle game using Python was presented. In this question, you will deal with a similar problem within the same concept.
Your goal now is to make the *computer* play the Wordle game and correctly guess the word in the shortest number of attempts.
As in the midterm, we have modularized the problem for you and provided a prepared template. By completing this template, you will solve the problem.

You cannot make any changes to this file.
To solve this question, you can either copy the code from each cell and paste it into your own notebook or save a copy in your Drive and work directly on that copy.

You can access the answer key for the midterm Wordle question from the link below.

https://colab.research.google.com/drive/12z3Bk4CSW4GWCpxX0WiftyzSdb3gTubp?usp=sharing


In this problem, the main function we will run will be the `optimize_wordle()` function, and the `Wordle()` function you wrote will now be used as a helper function.  
You need to completely change the structure of the Wordle function; it no longer needs to provide any colored output.  

For the best optimization, the Wordle function should return three pieces of information for the input we provide:  
- **list1**: Letters in the correct position.    
- **list2**: Letters in the wrong position.  
- **list3**: Letters not present in the word.  

Hint: You can return these three pieces of information as three separate lists.  
(For easier coding, you can structure these lists in different ways. (Nested lists might be useful.)


In the provided templates, we add the `pass` statement at the end of functions to avoid errors when the cell is executed since the functions will be completed later. After writing the templates, you need to remove the `pass` statement.

Additionally, in the provided templates, you will decide what inputs the functions should have. We will not include anything in these sections within the templates.


##IMPORTS
Here we will import some libraries into the notebook and write functions for generating random words.


In [ ]:
import requests
import random

# Ready Helper Functions

We give you ready functions to return all English words as a list.

If you do not understand the logic of these functions, this is not a problem, you just need to run the functions to solve the problem.

##WARNING:
 If you are running Jupyter Notebook in a local environment like Anaconda, you may get an error because you did not download the library files to your computer, in this case it is your responsibility to solve this problem. We recommend using Google Colab, the functions will work without errors in Google Colab.

##WARNING:
Do not change the functions or links here, it may cause you to get an error.

##WARNING:
Please run these cells while your computer is connected to the internet because it needs to download all the English words from an external source.

In [ ]:
def download_file_from_drive(file_url, colab_path):
    if "drive.google.com" in file_url:
        file_id = file_url.split('/d/')[1].split('/')[0]
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    else:
        raise ValueError("The provided URL is not a valid Google Drive link.")

    response = requests.get(download_url, stream=True)

    if response.status_code == 200:
        with open(colab_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"File successfully downloaded to {colab_path}")
    else:
        raise Exception("Failed to download the file. Check the URL or file permissions.")

colab_path = "/content/words.txt"
file_url = "https://drive.google.com/file/d/1cjCZ-5uAHVFLXLV6HuSM98_QblWaXrBZ/view?usp=share_link"
download_file_from_drive(file_url,colab_path)


WORDLIST_FILENAME = "words.txt"

def load_words():

  inFile = open(WORDLIST_FILENAME, 'r')
  line = inFile.readline()
  wordlist = line.split()
  print("  ", len(wordlist), "words loaded.")
  return wordlist

File successfully downloaded to /content/words.txt


## Helper Function Templates

In this question, the `wordle()` function compares the secretword and the guess, returning three lists ***(you may adjust the number of outputs if needed)***.  
To avoid complexity and for simplicity in naming, use the following names for these three lists throughout the assignment:  

- **list1**: Letters in the correct position.    
- **list2**: Letters in the wrong position.  
- **list3**: Letters not present in the word.

You need to ensure that the computer makes guesses that are the same length as the `secretword`.


 #### HINTS FOR THE HELPER FUNCTIONS

 `word_select()`: We had provided this function to you as ready-made in the midterm, but for this question, you need to make a small modification to its structure. Additionally, you will later use this function to generate a random guess.

 `cleanwordlist()`: In this function, you can update your word list to ensure that the computer makes guesses with the same length as the secretword.  

`updatelist1/2/3()`: If needed, you can use these three functions to update the lists that will be returned by the `wordle()` function.  

Of course, you can solve the problem without these functions, but modularizing the problem makes things easier.  

You can also define additional helper functions if you wish.




In [ ]:
def word_select(wordlist):
    return random.choice(wordlist)

In [ ]:
def updatelist1(guess, secretWord):
    matches = []  # Correct letters in correct positions
    i = 0         # Initialize index
    while i < len(secretWord):
        if guess[i] == secretWord[i]:  # if the letters match
            matches.append([guess[i], i])  # Save the letter and its position
        i += 1  # Move to the next index
    return matches

In [ ]:
def updatelist2(guess, secretWord, list1):
    wrong_positions = []  # letters in wrong position
    used_spots = []      # spots we already used

    # Remember used spots from list1
    i = 0
    while i < len(list1):
        used_spots.append(list1[i][1])
        i += 1

    # Look at each letter in our guess
    i = 0
    while i < len(guess):
        # Skip if we already used this spot
        if i not in used_spots:
            letter = guess[i]
            # Look for this letter in secret word
            j = 0
            while j < len(secretWord):
                # If we find the letter and spot isn't used
                if letter == secretWord[j] and j not in used_spots:
                    # Remember letter and its wrong spot
                    wrong_positions.append([letter, i])
                    used_spots.append(j)
                    break
                j += 1
        i += 1

    return wrong_positions

In [ ]:
def updatelist3(guess, secretWord, list1, list2):
    wrong_letters = []   # letters not in word
    right_letters = []   # letters we got right

    # Remember letters we got right
    i = 0
    while i < len(list1):
        right_letters.append(list1[i][0])
        i += 1

    i = 0
    while i < len(list2):
        right_letters.append(list2[i][0])
        i += 1

    # Check each letter in guess
    i = 0
    while i < len(guess):
        letter = guess[i]
        # If letter isn't in secret word and we haven't found it yet
        if letter not in secretWord and letter not in wrong_letters:
            wrong_letters.append(letter)
        # If letter is in word but we found too many of it
        elif letter not in right_letters and letter not in wrong_letters:
            wrong_letters.append(letter)
        i += 1

    return wrong_letters

In [ ]:
def cleanwordlist(wordlist, secretWord):
    # Make a new empty list for words with same length
    cleaned_list = []
    for word in wordlist:
        if len(word) == len(secretWord):
            cleaned_list.append(word)
    return cleaned_list

## `wordle()` Procedure

As explained above, the `wordle` function will now return three lists to provide information to the computer. Therefore, you need to completely change the structure of the function.  

**Extra Hint:** Do not define a `secretword` within the `wordle` function. Instead, compare a pre-determined `secretword` and `guess` within this function.  


In [ ]:
def wordle(guess,secretWord):
  """
  Compare a guess with secretWord and return three lists:
    - list1: Correct letters in correct positions
    - list2: Correct letters in wrong positions
    - list3: Letters not in word
  """
  # Get feedback about the guess using our helper functions
  list1 = updatelist1(guess, secretWord)
  list2 = updatelist2(guess, secretWord, list1)
  list3 = updatelist3(guess, secretWord, list1, list2)

  return list1, list2, list3

## `generate_guess()` Procedure

This function will generate a new guess based on the information provided by the wordle function.

For example, the information obtained from the previous guess includes:

- **list1**: Letters in the correct position.    
- **list2**: Letters in the wrong position.  
- **list3**: Letters not present in the word.

Using these lists, the function should refine its guesses and generate the next word accordingly.


That means if we have the information that the word contains the letter `'e'` but not in the 3rd position, this function must select a word that includes the letter `'e'` and ensures that `'e'` is not in the 3rd position.  

In this way, the function must choose a new word that adheres to all the information gathered so far.  

Additionally, the new guess must also comply with all the information obtained from previous guesses. The function should consistently refine its guesses based on accumulated knowledge and eliminate invalid options from the pool of possible words.

In [ ]:
def generate_guess(wordlist, list1, list2, list3, secretWord):
    possible_words = cleanwordlist(wordlist, secretWord)  # list of words with matching length
    newguess_words = []  # words that pass our checks

    # Check each possible word
    for word in possible_words:
        # Assume word is good until we prove it's not
        keeptheword = True

        # Check list1 (right letters, right spots)
        for item in list1:
            letter = item[0]    # Get the letter
            position = item[1]  # Get its position
            if word[position] != letter:
                keeptheword = False
                break

        # If word failed list1 check, try next word
        if not keeptheword:
            continue

        # Check list2 (right letters, wrong spots)
        for item in list2:
            letter = item[0]           # Get the letter
            wrong_position = item[1]   # Get position where it shouldn't be
            if letter not in word or word[wrong_position] == letter:
                keeptheword = False
                break

        # If word failed list2 check, try next word
        if not keeptheword:
            continue

        # Check list3 (wrong letters)
        for wrong_letter in list3:
            if wrong_letter in word:  # If this wrong letter appears in word
                keeptheword = False
                break

        # If word passed all checks, add it to newguess_words list
        if keeptheword:
            newguess_words.append(word)

    # Return a random newguess_word if we found any
    if newguess_words:
        return word_select(newguess_words)
    return None

## `optimize_wordle()` Procedure

The `optimize_wordle()` function is our main function, and it will be used to make the computer play the Wordle game. This is the function we will call to run the entire process.

#### HINTS

Set the target word as `secretword`.  
Ensure that the computer generates guesses of the same length as the `secretword`.  
Then, use a loop to sequentially generate a guess and send it to the `wordle` function. Continue this loop until the computer correctly guesses the word.  

Once the computer has guessed the word, print the number of attempts it took and the list of words it guessed.  
To make the function output more user-friendly, also display the computer's current guess in real time while the function is running.  


### Sample Output
    optimize_wordle()

\>>

      55900 words loaded.

    Secretword is :gong

    Computer will try to find this word...

    Computer's guess is: bite

    Computer's guess is: arms

    Computer's guess is: loop

    Computer's guess is: jock

    Computer's guess is: hong

    Computer's guess is: gong

    The computer guessed the word in 6 attempts.

    List of the computer's guesses:['bite', 'arms', 'loop', 'jock', 'hong', 'gong']

In [ ]:
def optimize_wordle():
    # Load words and pick secret word
    wordlist = load_words()
    secretWord = word_select(wordlist)

    print("\nSecretword is:", secretWord)
    print("\nComputer will try to find this word...")

    # To Keep track of guesses
    attempts = 0
    guessed_words = []

    # Store what we learn from each guess
    correct = []     # right letter, right spot / use of list1
    misplaced = []   # right letter, wrong spot / use of list2
    wrong = []       # wrong letters / use of list3

    while True:
        # For first guess, just pick any random word with same length
        if attempts == 0:
            guess = word_select(cleanwordlist(wordlist, secretWord))
        # For later guesses, use what we learned so far
        else:
            guess = generate_guess(wordlist, correct, misplaced, wrong, secretWord)

        attempts += 1
        guessed_words.append(guess)

        print("\nComputer's guess is:", guess)

        # If we found the word, we're done
        if guess == secretWord:
            print("\nThe computer guessed the word in", attempts, "attempts.")
            print("List of computer's guesses:", guessed_words)
            return attempts

        else:
            # Get feedback about our guess
            correct, misplaced, wrong = wordle(guess, secretWord)

# Run Main Function
optimize_wordle()

   55900 words loaded.

Secretword is: circumvent

Computer will try to find this word...

Computer's guess is: fallacious

Computer's guess is: reinducted

Computer's guess is: assurances

Computer's guess is: uncommonly

Computer's guess is: circumvent

The computer guessed the word in 5 attempts.
List of computer's guesses: ['fallacious', 'reinducted', 'assurances', 'uncommonly', 'circumvent']


5